In [1]:
from time import time
from model import S3VR
import numpy as np
from utils import *
from model import S3VR
import itertools
from sklearn.svm import SVR
from tqdm import tqdm
from sklearn.model_selection import KFold

In [2]:
labeled_X = np.load('labeled_data.npy')
unlabeled_X = np.load('unlabeled_data.npy')
y = np.load('labels.npy')

# info:

k-local: 5, 10, 15, 25

k-global: 10, 20, 30, 50

r: 0.5

beta: 10, 100, 1000

C: 0.1, 1, 10, 100

epsilon: 1e-2, 0.1, 0.5, 1

In [4]:
p = [[5, 10, 15, 25], [10, 20, 30, 50], [0.5], [10, 100, 1000], [0.1, 1, 10, 100],[1e-2, 0.1, 0.5, 1]]
ps = list(itertools.product(*p))

In [9]:
kf = KFold(n_splits=5)

res = np.zeros((5, len(ps)))
row = 0
s3vr = S3VR(None, None, None, None, None, None)
with tqdm(total=5 * len(ps)) as progress:
    for train, test in kf.split(labeled_X):
        X_train, X_test, y_train, y_test = labeled_X[train], labeled_X[test], y[train], y[test]
        columns = 0
        for k_local, k_global, r, beta, C, epsilon in ps:
            s3vr.k_local, s3vr.k_global, s3vr.r, s3vr.beta, s3vr.C, s3vr.epsilon = k_local, k_global, r, beta, C, epsilon
            s3vr.fit(X_train, y_train, unlabeled_X)
            preds = s3vr.predict(X_test)
            res[row, columns] = MCRMSE(y_test, preds.T)
            columns += 1
            progress.update(1)
        row += 1

100%|██████████| 20/20 [04:48<00:00, 14.41s/it]


In [10]:
np.save('res.npy', res)

ps[np.argsort(res.mean(axis=0))[0]]

(5, 10, 0.5, 10, 0.1, 0.1)